In [ ]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12

In [ ]:
import pandas as pd
from datasets import load_dataset
# Load the raw version of GoEmotions
df = load_dataset("google-research-datasets/go_emotions", "raw")
df = pd.DataFrame(df['train'])


In [ ]:
print(df.head() , df.shape , df.info(), df.describe)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211225 entries, 0 to 211224
Data columns (total 37 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   text                  211225 non-null  object 
 1   id                    211225 non-null  object 
 2   author                211225 non-null  object 
 3   subreddit             211225 non-null  object 
 4   link_id               211225 non-null  object 
 5   parent_id             211225 non-null  object 
 6   created_utc           211225 non-null  float64
 7   rater_id              211225 non-null  int64  
 8   example_very_unclear  211225 non-null  bool   
 9   admiration            211225 non-null  int64  
 10  amusement             211225 non-null  int64  
 11  anger                 211225 non-null  int64  
 12  annoyance             211225 non-null  int64  
 13  approval              211225 non-null  int64  
 14  caring                211225 non-null  int64  
 15  

In [ ]:
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer
import tensorflow as tf

# Load the raw version of GoEmotions
dataset = load_dataset("google-research-datasets/go_emotions", "raw")

# Convert to DataFrame for easier manipulation
df = pd.DataFrame(dataset['train'])

# Define label mappings
positive_emotions = [
    'admiration', 'amusement', 'approval', 'caring', 'excitement', 'gratitude',
    'joy', 'love', 'optimism', 'pride', 'relief', 'surprise'
]
negative_emotions = [
    'anger', 'annoyance', 'disappointment', 'disapproval', 'disgust', 'embarrassment',
    'fear', 'grief', 'nervousness', 'remorse', 'sadness'
]
neutral_emotions = ['neutral']

actionable_emotions = [
    'approval', 'caring', 'excitement', 'fear', 'gratitude', 'love',
    'pride', 'relief', 'surprise'
]

# Map emotions to sentiment
def map_sentiment(row):
    if any(row[emotion] == 1 for emotion in positive_emotions):
        return "positive"
    elif any(row[emotion] == 1 for emotion in negative_emotions):
        return "negative"
    else:
        return "neutral"

# Map emotions to actionable/non-actionable intents
def map_intent(row):
    return "actionable" if any(row[emotion] == 1 for emotion in actionable_emotions) else "non-actionable"

# Add sentiment and intent columns
df['sentiment'] = df.apply(map_sentiment, axis=1)
df['intent'] = df.apply(map_intent, axis=1)

# Preprocess text: Remove extra spaces, lowercase, etc.
def preprocess_text(text):
    text = text.lower().strip()
    text = text.replace("\n", " ")  # Replace newlines
    return text

df['text'] = df['text'].apply(preprocess_text)

# Split dataset
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Load RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Tokenization function
def tokenize_data(dataframe):
    return tokenizer(
        list(dataframe['text']),
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="tf"
    )

# Tokenize train, validation, and test sets
train_tokens = tokenize_data(train_df)
val_tokens = tokenize_data(val_df)
test_tokens = tokenize_data(test_df)


## Go emotions

In [ ]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.6 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizer
from datasets import load_dataset
from sklearn.metrics import classification_report
import numpy as np

In [ ]:
# Load GoEmotions dataset
dataset = load_dataset("go_emotions")

# Preprocess the data
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

def preprocess_labels(example):
    multi_hot_labels = np.zeros(28)  # Assuming 28 possible labels in GoEmotions
    for label in example['labels']:
        multi_hot_labels[label] = 1
    example['labels'] = multi_hot_labels.tolist()
    return example

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.40k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/350k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/347k [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Tokenize and preprocess the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.map(preprocess_labels)

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

In [ ]:
# Prepare TensorFlow datasets
train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["labels"],
    shuffle=True,
    batch_size=32  # Optimized batch size for memory
)
validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["labels"],
    shuffle=False,
    batch_size=32
)


/usr/local/lib/python3.11/dist-packages/datasets/arrow_dataset.py:405: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [ ]:
# Enable mixed precision
policy = tf.keras.mixed_precision.Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)

In [ ]:
# Load pre-trained BERT model with a classification head
with tf.device('/GPU:0'):
    model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=28, problem_type="multi_label_classification")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Compile the model with an optimized learning rate schedule
learning_rate = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=5e-5,
    end_learning_rate=1e-6,
    decay_steps=10000
)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = [
    tf.keras.metrics.Precision(name="precision"),
    tf.keras.metrics.Recall(name="recall"),
    tf.keras.metrics.AUC(name="auc"),
    tf.keras.metrics.BinaryAccuracy(name="accuracy")
]
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
# Train the model with callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint(filepath="model_checkpoint", save_best_only=True)
]

In [ ]:
with tf.device('/GPU:0'):
    model.fit(train_dataset, validation_data=validation_dataset, epochs=5, callbacks=callbacks,validation_split=0.2)

# Evaluation function
def evaluate_model(model, dataset):
    with tf.device('/GPU:0'):
        predictions = model.predict(dataset)
    logits = predictions.logits
    preds = (tf.sigmoid(logits) > 0.5).numpy()
    labels = np.concatenate([label for _, label in dataset], axis=0)
    print(classification_report(labels, preds, target_names=[f'label_{i}' for i in range(28)]))

# Evaluate the model
evaluate_model(model, validation_dataset)

Epoch 1/5
1357/1357 [==============================] - 578s 383ms/step - loss: 0.1264 - precision: 0.7780 - recall: 0.1499 - auc: 0.6178 - accuracy: 0.9625 - val_loss: 0.0939 - val_precision: 0.8204 - val_recall: 0.2549 - val_auc: 0.6759 - val_accuracy: 0.9664
Epoch 2/5
1357/1357 [==============================] - 519s 382ms/step - loss: 0.0846 - precision: 0.8157 - recall: 0.3527 - auc: 0.7218 - accuracy: 0.9694 - val_loss: 0.0854 - val_precision: 0.7597 - val_recall: 0.4078 - val_auc: 0.7408 - val_accuracy: 0.9697
Epoch 3/5
1357/1357 [==============================] - 461s 340ms/step - loss: 0.0682 - precision: 0.8509 - recall: 0.4866 - auc: 0.7847 - accuracy: 0.9748 - val_loss: 0.0868 - val_precision: 0.7243 - val_recall: 0.4447 - val_auc: 0.7496 - val_accuracy: 0.9696
Epoch 4/5
170/170 [==============================] - 23s 112ms/step
              precision    recall  f1-score   support

     label_0       0.76      0.70      0.73       488
     label_1       0.77      0.83      0

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/TEMini/GoemotionsBert
!ls

/content/drive/MyDrive/TEMini/GoemotionsBert


In [ ]:
# prompt: Save the model to .pkl  file in the current root

import pickle

# Save the model to a .pkl file
with open('emotion_model.pkl', 'wb') as file:
    pickle.dump(model, file)


/usr/local/lib/python3.11/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [ ]:
# Inference pipeline
def predict(text):
    inputs = tokenizer(text, return_tensors="tf", padding="max_length", truncation=True, max_length=128)
    with tf.device('/GPU:0'):
        outputs = model(inputs)
    logits = outputs.logits
    predictions = (tf.sigmoid(logits) > 0.5).numpy()
    return predictions

# Example usage
text = "I am feeling very happy and content today!"
print(predict(text))


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
1357/1357 [==============================] - 578s 393ms/step - loss: 0.1252 - precision: 0.7818 - recall: 0.1641 - auc: 0.6194 - accuracy: 0.9629 - val_loss: 0.0925 - val_precision: 0.7961 - val_recall: 0.2981 - val_auc: 0.6956 - val_accuracy: 0.9673
Epoch 2/5
1357/1357 [==============================] - 519s 382ms/step - loss: 0.0843 - precision: 0.8143 - recall: 0.3576 - auc: 0.7250 - accuracy: 0.9696 - val_loss: 0.0852 - val_precision: 0.7802 - val_recall: 0.3672 - val_auc: 0.7289 - val_accuracy: 0.9691
Epoch 3/5
1357/1357 [==============================] - 482s 355ms/step - loss: 0.0679 - precision: 0.8491 - recall: 0.4931 - auc: 0.7873 - accuracy: 0.9750 - val_loss: 0.0868 - val_precision: 0.7160 - val_recall: 0.4469 - val_auc: 0.7519 - val_accuracy: 0.9693
Epoch 4/5
170/170 [==============================] - 23s 119ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

     label_0       0.73      0.74      0.73       488
     label_1       0.78      0.82      0.80       303
     label_2       0.74      0.29      0.41       195
     label_3       0.54      0.05      0.09       303
     label_4       0.50      0.24      0.32       397
     label_5       0.72      0.31      0.44       153
     label_6       0.70      0.25      0.37       152
     label_7       0.52      0.47      0.49       248
     label_8       0.73      0.42      0.53        77
     label_9       0.00      0.00      0.00       163
    label_10       0.55      0.28      0.37       292
    label_11       0.70      0.22      0.33        97
    label_12       0.00      0.00      0.00        35
    label_13       0.64      0.07      0.13        96
    label_14       0.82      0.41      0.55        90
    label_15       0.96      0.88      0.92       358
    label_16       0.00      0.00      0.00        13
    label_17       0.76    

In [ ]:
eval = evaluate_model(model, validation_dataset)

170/170 [==============================] - 22s 127ms/step
              precision    recall  f1-score   support

     label_0       0.73      0.74      0.73       488
     label_1       0.78      0.82      0.80       303
     label_2       0.74      0.29      0.41       195
     label_3       0.54      0.05      0.09       303
     label_4       0.50      0.24      0.32       397
     label_5       0.72      0.31      0.44       153
     label_6       0.70      0.25      0.37       152
     label_7       0.52      0.47      0.49       248
     label_8       0.73      0.42      0.53        77
     label_9       0.00      0.00      0.00       163
    label_10       0.55      0.28      0.37       292
    label_11       0.70      0.22      0.33        97
    label_12       0.00      0.00      0.00        35
    label_13       0.64      0.07      0.13        96
    label_14       0.82      0.41      0.55        90
    label_15       0.96      0.88      0.92       358
    label_16       0.00

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
def predict(text):
    inputs = tokenizer(text, return_tensors="tf", padding="max_length", truncation=True, max_length=128)
    with tf.device('/GPU:0'):
        outputs = model(inputs)
    logits = outputs.logits
    probabilities = tf.sigmoid(logits).numpy()[0]
    top_indices = np.argsort(probabilities)[-3:][::-1]
    top_labels_and_scores = [(f'label_{i}', probabilities[i]) for i in top_indices]
    return top_labels_and_scores

# Example usage
text = "I am feeling very happy and content today!"
print(predict(text))

[('label_17', 0.84), ('label_13', 0.07355), ('label_15', 0.05533)]


In [ ]:
print(predict("i feel that i am very stamped with work even when there is not much work. i dont know why it is so bad"))

[('label_6', 0.709), ('label_7', 0.08124), ('label_25', 0.0667)]


In [ ]:
model.summary()

Model: "tf_bert_for_sequence_classification_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_259 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  21532     
                                                                 
Total params: 109503772 (417.72 MB)
Trainable params: 109503772 (417.72 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
